In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "CHASEC123"
db = AnimalShelter(username, password)

# Read data from MongoDB
df = pd.DataFrame.from_records(db.read({}))

# Drop '_id' column to avoid issues with ObjectID. 
# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)

if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

    
# Initialize the app
app = JupyterDash(__name__)

# Encode the first logo image for display
image_filename = 'GraziosoSalvareLogo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Encode the second background image for display
image_filename1 = 'bg1.png'  # replace with your own image
encoded_image1 = base64.b64encode(open(image_filename1, 'rb').read())

#########################
# Dashboard Layout / View
#########################

app.layout = html.Div([
    # Hidden div for callbacks
    html.Div(id='hidden-div', style={'display': 'none'}),
    
    # Top logo (background image)
    html.Center(
        html.Img(
            id='background-image',
            src=f'data:image/png;base64,{encoded_image1.decode()}',
            alt='Background Image',
            style={'width': '200px', 'height': 'auto'}
        )
    ),
    
    # Dashboard headline
    html.Center(html.B(html.H1('Chase Carter CS340 Project 2 Dashboard'))),
    
    # Second logo under the headline (Grazioso Salvare Logo)
    html.Center(
        html.Img(
            id='logo-image',
            src=f'data:image/png;base64,{encoded_image.decode()}',
            alt='Grazioso Salvare Logo',
            style={'width': '200px', 'height': 'auto'}
        )
    ),
    
    html.Hr(),
    
    # Filter options
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
    ),
    
    html.Hr(),
    
    # DataTable
    dt.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    
    html.Br(),
    html.Hr(),
    
    # Graph and map layout
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6'),
                 html.Div(id='map-id', className='col s12 m6')
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Radio button filters
@app.callback([Output('datatable-id', 'data'),
               Output('datatable-id', 'columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
    if filter_type == 'water':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }
    elif filter_type == 'mount':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                              "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }
    elif filter_type == 'disaster':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                              "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        }
    else:  # Reset
        query = {}

    df = pd.DataFrame.from_records(db.read(query))
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')

    return data, columns


# Display a histogram of breeds
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    #]
    
    if viewData is None or not viewData:
        return "No data available for visualization"

    dff = pd.DataFrame.from_dict(viewData)
    if 'breed' in dff.columns:
        return [
            dcc.Graph(
                figure=px.histogram(dff, x='breed', title='Available Dogs by Breed')
            )
        ]
    else:
        return "No 'breed' column in data"


# Update the geo-location chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData:
        return "No data available for map"
    if index is None or not index:
        return "No selection made"

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    if 'location_lat' in dff.columns and 'location_long' in dff.columns:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                    dl.Tooltip(dff.iloc[row]['breed'] if 'breed' in dff.columns else "Unknown"),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row]['name'] if 'name' in dff.columns else "Unknown")
                    ])
                ])
            ])
        ]
    else:
        return "No location data available"


if __name__ == '__main__':
    app.run_server(debug=False)

